In [1]:
# Pandas, numpy, matplotlib and seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# RiotWatcher
from riotwatcher import LolWatcher, ApiError

# OS tools
import os
import shutil
import json
import sys

# Custom scripts
from extract_players_performance import extract_players_performance
from remove_perks import remove_perks
from cleaner import replace_champ_names_with_tags
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
with open('../champion_data/champion.json','r',encoding='utf-8') as f :
    champions = json.load(f)



def add(old,new_df) :
    appended_df = pd.concat([old,new_df],ignore_index=True)
    return appended_df

def champ_name_replacer(champ_name) :
    champ_data = champions['data']
    return champ_data[champ_name]['tags'][0]

In [3]:
df1 = pd.read_csv('../data/Jedı07_match_history/matches/match_NA1_4034399637/ml_data.csv',delimiter=',')
df2 = pd.read_csv('../data/Jedı07_match_history/matches/match_NA1_4034434056/ml_data.csv',delimiter=',')

df = pd.DataFrame()


df = add(df,df1)
df = add(df,df2)


columns = list(df.columns)
champ_name_columns = [c for c in columns if c.startswith('team_comp') or c.startswith('dmg_carry') or c.startswith('obj_carry')]


for c in champ_name_columns :
    df[c] = df[c].apply(lambda x : champ_name_replacer(x))

print(f"dataframe sample:\n{df.head()}\n")


unique_champ_tags = df[champ_name_columns].stack().unique()
dummy_dict = {}
for k in champ_name_columns :
    dummy_dict[k] = unique_champ_tags.copy()
dummy_df = pd.DataFrame(dummy_dict)

extra_rows = dummy_df.shape[0]

df = add(df,dummy_df)
print(df.loc[:1,champ_name_columns].values)

x = df[champ_name_columns]
ohe = OneHotEncoder()
x = ohe.fit_transform(x).toarray()


total_size = len(x)
x = x[:total_size-extra_rows]

print(x)

dataframe shape:
(2, 51)

dataframe columns:
Index(['total_gold_earned_0', 'total_gold_spent_0', 'total_baron_kills_0',
       'total_dragon_kills_0', 'total_inhibitor_kils_0', 'total_kills_0',
       'total_deaths_0', 'total_damage_dealt_to_champions_0',
       'total_damage_dealt_to_objectives_0', 'total_damage_taken_0',
       'average_vision_score_0', 'total_wards_placed_0',
       'average_creep_score_0', 'average_champion_experience_0', 'dmg_carry_0',
       'obj_carry_0', 'team_comp_0_champ_1', 'team_comp_0_champ_2',
       'team_comp_0_champ_3', 'team_comp_0_champ_4', 'team_comp_0_champ_5',
       'total_gold_earned_1', 'total_gold_spent_1', 'total_baron_kills_1',
       'total_dragon_kills_1', 'total_inhibitor_kils_1', 'total_kills_1',
       'total_deaths_1', 'total_damage_dealt_to_champions_1',
       'total_damage_dealt_to_objectives_1', 'total_damage_taken_1',
       'average_vision_score_1', 'total_wards_placed_1',
       'average_creep_score_1', 'average_champion_experie

In [4]:
unique_champ_tags = df[champ_name_columns].stack().unique()
print(unique_champ_tags)
dummy_dict = {}
for k in champ_name_columns :
    dummy_dict[k] = unique_champ_tags.copy()
dummy_df = pd.DataFrame(dummy_dict)
extra_rows = dummy_df.shape[0]
print(dummy_df)
print('Number of extra rows = ',extra_rows)

['Marksman' 'Fighter' 'Mage' 'Support' 'Assassin' 'Tank']
  dmg_carry_0 obj_carry_0 team_comp_0_champ_1 team_comp_0_champ_2  \
0    Marksman    Marksman            Marksman            Marksman   
1     Fighter     Fighter             Fighter             Fighter   
2        Mage        Mage                Mage                Mage   
3     Support     Support             Support             Support   
4    Assassin    Assassin            Assassin            Assassin   
5        Tank        Tank                Tank                Tank   

  team_comp_0_champ_3 team_comp_0_champ_4 team_comp_0_champ_5 dmg_carry_1  \
0            Marksman            Marksman            Marksman    Marksman   
1             Fighter             Fighter             Fighter     Fighter   
2                Mage                Mage                Mage        Mage   
3             Support             Support             Support     Support   
4            Assassin            Assassin            Assassin    Assassin

In [5]:
df = add(df,dummy_df)
print(df.loc[:1,champ_name_columns].values)

x = df[champ_name_columns]
ohe = OneHotEncoder()
x = ohe.fit_transform(x).toarray()


total_size = len(x)
x = x[:total_size-extra_rows]

print(x)

[['Marksman' 'Fighter' 'Fighter' 'Fighter' 'Mage' 'Marksman' 'Mage'
  'Marksman' 'Fighter' 'Mage' 'Fighter' 'Marksman' 'Marksman' 'Support']
 ['Mage' 'Fighter' 'Support' 'Fighter' 'Mage' 'Marksman' 'Mage' 'Fighter'
  'Marksman' 'Fighter' 'Fighter' 'Assassin' 'Marksman' 'Tank']]
[[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0